# Chapter7: Dictionary Tricks

## 7.1 Dictionary Default Values

“It’s inefficient because it queries the dictionary twice.

It’s verbose since part of the greeting string is repeated, for example.

It’s not Pythonic—the official Python documentation specifically recommends an “easier to ask for forgiveness than permission” (EAFP) coding style for these situations:

“This common Python coding style assumes the existence of valid keys or attributes and catches exceptions if the assumption proves false.”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [1]:
name_for_userid = {
    382: 'Alice',
    950: 'Bob',
    590: 'Dilbert',
}

def greeting(userid):
    if userid in name_for_userid:
        return f'Hi {name_for_userid[userid]}!'
    else:
        return 'Hi there!'

In [2]:
greeting(382)

'Hi Alice!'

In [3]:
greeting(3333)

'Hi there!'

In [4]:
def greeting(userid):
    try:
        return f'Hi {name_for_userid[userid]}'
    except KeyError:
        return 'Hi there'

In [5]:
greeting(382)

'Hi Alice'

In [6]:
greeting(3333)

'Hi there'

In [10]:
def greeting(userid):
    return f"Hi {name_for_userid.get(userid, 'there')}!"

In [11]:
greeting(950)

'Hi Bob!'

In [12]:
greeting(333)

'Hi there!'

### Key Takeaways

- Avoid explicit key in dict checks when testing for membership.
- EAFP-style exception handling or using the built-in get() method is preferable.
- In some cases, the collections.defaultdict class from the standard library can also be helpful.

## 7.2 Sorting Dictionaries for Fun and Profit

“Python dictionaries don’t have an inherent order. You can iterate over them just fine but there’s no guarantee that iteration returns the dictionary’s elements in any particular order (although this is changing with Python 3.6).”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [14]:
xs = {'a': 4, 'c': 2, 'b': 3, 'd': 1}
xs

{'a': 4, 'c': 2, 'b': 3, 'd': 1}

In [15]:
sorted(xs.items())

[('a', 4), ('b', 3), ('c', 2), ('d', 1)]

In [16]:
sorted(xs.items(), key=lambda x: x[1])

[('d', 1), ('c', 2), ('b', 3), ('a', 4)]

In [17]:
import operator
sorted(xs.items(), key=operator.itemgetter(1))

[('d', 1), ('c', 2), ('b', 3), ('a', 4)]

In [18]:
sorted(xs.items(), key=lambda x: x[1], reverse=True)

[('a', 4), ('b', 3), ('c', 2), ('d', 1)]

### Key Takeaways

- When creating sorted “views” of dictionaries and other collections, you can influence the sort order with a key func.
- Key funcs are an important concept in Python. The most frequently used ones were even added to the operator module in the standard library.
- Functions are first-class citizens in Python. This is a powerful feature you’ll find used everywhere in the language.

## 7.3 Emulating Switch/Case Statements With Dicts

In [19]:
def dispatch_if(operator, x, y):
    if operator == 'add':
        return x + y
    elif operator == 'sub':
        return x - y
    elif operator == 'mul':
        return x * y
    elif operator == 'div':
        return x / y

In [21]:
dispatch_if('mul', 2, 8)

16

In [22]:
dispatch_if('unknown', 2, 8)

In [23]:
def dispatch_dict(operator, x, y):
    return {
        'add': lambda: x + y,
        'sub': lambda: x - y,
        'mul': lambda: x * y,
        'div': lambda: x / y,
    }.get(operator, lambda: None)()

In [24]:
dispatch_dict('mul', 2, 8)

16

In [25]:
dispatch_dict('unknown', 2, 8)

### Key Takeaways

- Python doesn’t have a switch/case statement. But in some cases you can avoid long if-chains with a dictionary-based dispatch table.
- Once again Python’s first-class functions prove to be a powerful tool. But with great power comes great responsibility.

## 7.4 The Craziest Dict Expression in the West

In [27]:
{True: 'yes', 1: 'no', 1.0: 'maybe'}

{True: 'maybe'}

Behind the scenes...

In [28]:
xs = dict()
xs[True] = 'yes'
xs[1] = 'no'
xs[1.0] = 'maybe'
xs

{True: 'maybe'}

In [29]:
True == 1 == 1.0

True

In [30]:
hash(True), hash(1), hash(1.0)

(1, 1, 1)

### Key Takeaways

- Dictionaries treat keys as identical if their `__eq__` comparison result says they’re equal and their hash values are the same.
- Unexpected dictionary key collisions can and will lead to surprising results.

## 7.5 So Many Ways to Merge Dictionaries

In [34]:
xs = {'a': 1, 'b': 2}
ys = {'b': 3, 'c': 4}

In [35]:
zs = {}
zs.update(xs)
zs.update(ys)

In [36]:
zs

{'a': 1, 'b': 3, 'c': 4}

“Starting with Python 3.5, the `**`-operator became more flexible. So in Python 3.5+ there’s another—and arguably prettier—way to merge an arbitrary number of dictionaries.

This expression has the exact same result as a chain of update() calls. Keys and values are set in a left-to-right order, so we get the same conflict resolution strategy: the right-hand side takes priority, and a value in ys overrides any existing value under the same key in xs.”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [37]:
zs = {**xs, **ys}

In [38]:
zs

{'a': 1, 'b': 3, 'c': 4}

### Key Takeaways

- In Python 3.5 and above you can use the `**`-operator to merge multiple dictionary objects into one with a single expression, overwriting existing keys left-to-right.
- To stay compatible with older versions of Python, you might want to use the built-in dictionary update() method instead.

## 7.6 Dictionary Pretty-Printing

In [39]:
mapping = {'a': 23, 'b': 42, 'c': 0xc0ffee}
str(mapping)

"{'a': 23, 'b': 42, 'c': 12648430}"

In [40]:
import json
json.dumps(mapping, indent=4, sort_keys=True)

'{\n    "a": 23,\n    "b": 42,\n    "c": 12648430\n}'

“While this looks nice and readable, it isn’t a perfect solution. Printing dictionaries with the json module only works with dicts that contain primitive types”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [41]:
json.dumps({all: 'yup'})

TypeError: keys must be a string

In [42]:
mapping['d'] = {1, 2, 3}
json.dumps(mapping)

TypeError: Object of type 'set' is not JSON serializable

In [43]:
import pprint
pprint.pprint(mapping)

{'a': 23, 'b': 42, 'c': 12648430, 'd': {1, 2, 3}}


“However, compared to json.dumps(), it doesn’t represent nested structures as well visually. Depending on the circumstances, this can be an advantage or a disadvantage. I occasionally use json.dumps() to print dictionaries because of the improved readability and formatting, but only if I’m sure they’re free of non-primitive data types.”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

### Key Takeaways

- The default to-string conversion for dictionary objects in Python can be difficult to read.
- The pprint and json module are “higher-fidelity” options built into the Python standard library.
- Be careful with using json.dumps() and non-primitive keys and values as this will trigger a TypeError.